In [1]:
from sqlalchemy import create_engine,text
import pandas as pd
import json,os,re

import tiktoken
from numpyencoder import NumpyEncoder

import string
from langchain.text_splitter import RecursiveCharacterTextSplitter





In [2]:
designation_dict={'chief executive officer': 1,
"founder & managing director":1,
 'chief executive': 1,
 'ceo': 1,
 'managing partner': 1,
 'managing director': 1,
 'founder': 1,
 'owner': 1,
 'chairman': 1,
 'director': 1,
 'executive director': 1,
 'co-founder': 1,
 'cofounder': 1,
 'co founder': 1,
 'group managing director': 1,
 }



def LengthFormat2(data,ignor_list_name):
    result={}
    
    keys=[key for key in data.keys()]
    try:
        if data["name"].lower()  in ignor_list_name:
            return result
            
    except:
        return {}
    else:

        if "name" in keys and "designation" in keys:
            try:
                designation=data["designation"].lower()
                for role in designation_dict:
                    
                    if role in designation:
                        if data["designation"] in result:
                            
                            result[data["designation"].lower()].add(data["name"].lower())
                        else:
                            result[data["designation"].lower()]={data["name"].lower()}
            except:pass
        if "name" in keys and "title" in keys:
            try:
                
                designation=data["title"].lower()
                for role in designation_dict:
                    if role in designation:
                        if data["title"] in result:
                            result[data["title"].lower()].add(data["name"])
                        else:
                            result[data["title"].lower()]={data["name"]}
            except:pass
    return result



def ExactDesignationMatch(data,ignor_list_name,result):
    for designation , name in data.items():
        
        try:
            name=name.lower()
        
            designation=designation.lower()
            designation_dict[designation]
            if name  in ignor_list_name:
                pass
            else:
                
                if designation in result:
                    result[designation].add(name)
                else:
                    result[designation]={name}
                
        except:
            pass
    for name,designation in data.items():
        
        try:
            name=name.lower()
        
            designation=designation.lower()
            designation_dict[designation]
            if name  in ignor_list_name:
                pass
            else:
                
                if designation in result:
                    result[designation].add(name)
                else:
                    result[designation]={name}
        except:
            pass
    return result

def PartialCheckDesignation(data,ignor_list_name,result):
    ## IN checking
    for name , designation in data.items():
        try:
            name=name.lower()
            designation=designation.lower()
            for role in designation_dict:
                if role in designation:
                    if name  in ignor_list_name:
                        pass
                    else:

                        if designation in result:
                            
                            result[designation].add(name)
                        else:
                            result[designation]={name}
        except:
            pass
                    
    for designation , name in data.items():
        try:
            name=name.lower()
            designation=designation.lower()
            for role in designation_dict:
                if role in designation:
                    if name  in ignor_list_name:
                        pass
                    else:
                        if designation in result:
                            result[designation].add(name)
                        else:
                            result[designation]={name}
        except:
            pass
    return result
    
    
def Extract(data):
    ignor_list_name=["name","title","example","unknown","not specified","n/a","ceo","founder","owner","president","partner","director","managing"," director","cheif","officer","unspecified","not mentioned","md/ceo","designation","chairman","not provided","value","mission","vision","executive","m.d"]
    
    result={}
    if len(data)==2:
        
        keys=[key.lower() for key in data.keys() if key!=None]
 
        if "name" in keys and ("designation" in keys or "title" in keys) :
            result=LengthFormat2(data,ignor_list_name)
      
    
    
    result=ExactDesignationMatch(data,ignor_list_name,result)
    result=PartialCheckDesignation(data,ignor_list_name,result)
    
    for people_designation,names_list in result.items():
        key1=people_designation.lower()
        pople_name_update=[]
        for role in designation_dict:
            if role in key1:
                for ele in names_list:
                    if ele in ignor_list_name or role in ele:
                        pass
                    else:
                        flag=True
                        for item in ignor_list_name:
                            if item in ele:
                                flag=False
                                break
                        if flag:
                            if ele not in pople_name_update and (ele!="in" or ele!="the" or ele!="not" or ele!="of"):
                                pople_name_update.append(ele)
            result[people_designation]=pople_name_update
    return result

In [3]:
# Set up the connection parameters
db_username = 'postgres'
db_password = 'postgres321123'
db_host = 'scrape.cpf8ws5bn2fs.ap-south-1.rds.amazonaws.com'
db_port = '5432'
db_name = 'tempuse'
schema_name = 'new_ceo'

# Create the database engine with the specified schema
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}?options=-csearch_path%3D{schema_name}')

In [4]:
RawResultChatGpt={}
ErrorParent={}

In [5]:
with engine.connect() as conn:
    result=conn.execute(text("select parent ,childlink,raw_chatgpt_result from raw_ceo_data"))
    for row in result:
        child_data=json.loads(row[2])
        if row[0] in RawResultChatGpt:
            try:
                raw_result=json.loads(child_data["raw-result"])
                RawResultChatGpt[row[0]]={**RawResultChatGpt[row[0]],**{row[1]:raw_result}}
            except:
                pass
        else:
            try:
                raw_result=json.loads(child_data["raw-result"])
                RawResultChatGpt[row[0]]={row[1]:raw_result}
            except :
                pass
                
        # print(row[0],json.loads(row[2]))

In [6]:
len(RawResultChatGpt)

12397

In [7]:
with engine.connect() as conn:
    result=conn.execute(text("select parent ,childlink,raw_chatgpt_result from priority_website"))
    for row in result:
        child_data=json.loads(row[2])
        if row[0] in RawResultChatGpt:
            try:
                raw_result=json.loads(child_data["raw-result"])
                RawResultChatGpt[row[0]]={**RawResultChatGpt[row[0]],**{row[1]:raw_result}}
            except:
                pass
        else:
            try:
                raw_result=json.loads(child_data["raw-result"])
                RawResultChatGpt[row[0]]={row[1]:raw_result}
            except :
                ErrorParent[row[0]]=1
                
        # print(row[0],json.loads(row[2]))
len(ErrorParent)

151

check not Found 

In [8]:
NotFound={}
with engine.connect() as conn:
    result=conn.execute(text("select parent from priority_website"))
    for row in result:
        try:
            RawResultChatGpt[row[0]]
        except:
            NotFound[row[0]]=1
len(NotFound)
    

36

In [9]:
websiteChildlinkCeoName={}

In [10]:
def ComputeNameDesignationMap(nameDesignationmap,temp_parse,link)->dict:
    for designation,list_name in temp_parse.items():
        for people in list_name:
            if people in nameDesignationmap:
                nameDesignationmap[people]["designation"].add(designation)
                nameDesignationmap[people]["childlink"].add(link)
            else:
                nameDesignationmap[people]={"designation":{designation},"childlink":{link}}
    return nameDesignationmap

In [11]:
i=0
for parent ,childMap in RawResultChatGpt.items():
    # if parent!="20microns.com":
    #     continue
    
    nameDesignationmap={}
    for link ,value in childMap.items():            
        if type(value)==dict:
            temp_parse=Extract(value)
            
            temp=ComputeNameDesignationMap(nameDesignationmap,temp_parse,link)
            nameDesignationmap={**nameDesignationmap,**temp}

           
        elif type(value)==list:
            for each_batch in value:
                temp_parse=Extract(each_batch)
                temp=ComputeNameDesignationMap(nameDesignationmap,temp_parse,link)
                nameDesignationmap={**nameDesignationmap,**temp}

   
    websiteChildlinkCeoName[parent]=nameDesignationmap
    i+=1
    
i,len(websiteChildlinkCeoName)

(14186, 14186)

In [12]:
update_websiteChildlinkCeoName={
    "parent":[],
    "people_name":[],
    "designation":[],
    "childlink":[]
}

In [13]:
for parent,ceo_data in websiteChildlinkCeoName.items():                    
    for name ,ceo_data in ceo_data.items():
        if name:
            update_websiteChildlinkCeoName["designation"].append(",".join(ceo_data["designation"]))
            update_websiteChildlinkCeoName["childlink"].append(",".join(ceo_data["childlink"]))
            update_websiteChildlinkCeoName["people_name"].append(name)
            update_websiteChildlinkCeoName["parent"].append(parent)
len(update_websiteChildlinkCeoName)

4

In [14]:
# pd.DataFrame(update_websiteChildlinkCeoName).to_sql("priority_website_result",con=engine,index=False,if_exists="replace")
df=pd.DataFrame(update_websiteChildlinkCeoName)
df.shape


(42083, 4)

In [15]:
# RawResultChatGpt["galaxystamping.com"]
df[df["parent"]=='20microns.com'].head(20)
# ErrorParent["signatureglobal.in"]
# websiteChildlinkCeoName["signatureglobal.in"]

,parent,people_name,designation,childlink
21,20microns.com,late chandresh s. parikh,founder chairman,20microns.com/founder-chairman
22,20microns.com,rajesh c. parikh,chairman & managing director,20microns.com/leadership-team
23,20microns.com,atil c. parikh,ceo & managing director,20microns.com/leadership-team
24,20microns.com,sejal parikh,non-executive woman director,20microns.com/leadership-team
25,20microns.com,atul patel,independent director,20microns.com/leadership-team
26,20microns.com,ramkisan devidayal,independent director,20microns.com/leadership-team
27,20microns.com,jaideep verma,independent director,20microns.com/leadership-team
28,20microns.com,ajay ranka,independent director,20microns.com/leadership-team


In [16]:
df["parent"].describe()

count         42083
unique        10617
top       ieema.org
freq             71
Name: parent, dtype: object

In [17]:
df[df["parent"]=='galaxystamping.com']

,parent,people_name,designation,childlink
35406,galaxystamping.com,chintan d sitapara,founder & managing director,galaxystamping.com/?tag=management
35407,galaxystamping.com,dharmshibhai sitapara,chairman,galaxystamping.com/?tag=management
35408,galaxystamping.com,vishwadeep bose,executive director,galaxystamping.com/?tag=management


In [18]:
df.to_sql("Ceo_Data_2",con=engine,index=False,if_exists='replace')

83

In [19]:
# json_records = df.to_json(orient='records')
# json_records=json.loads(json_records)

In [20]:
# type(json_records)

In [21]:
# len(json_records)

In [22]:
# from MongoConnection import establish_mongo_connection
# client=establish_mongo_connection()

# client=client["ceo"]["PreFinalCeoData"]
# client.insert_many(json_records)


<h1> End 

In [23]:
# leadershipmap=leadershipmap={"leader": 1, "team": 1, "management": 1, "executive": 1, "member": 1, "staff": 1, "about": 1, "employee": 1, "people": 1, "crew": 1, "board": 1, "director": 1, "partner": 1, "governance": 1, "corporate-governance": 1, "leadership": 1, "meet-the-team": 1, "company": 1, "profile": 1, "bios": 1, "our": 1, "vision": 1, "ceo's": 1, "directors": 1, "corporate": 1, "mission": 1, "meet": 1, "principle": 1, "ceo": 1, "founder": 1, "biography": 1, "succession": 1, "strategies": 1, "approach": 1, "excellence": 1, "talent": 1, "culture": 1, "strategy": 1, "who": 1, "committee": 1, "success": 1, "growth": 1, "mentorship": 1, "inspiration": 1, "transformation": 1, "accountability": 1, "diversity": 1, "inclusion": 1, "adaptability": 1, "resilience": 1, "trust": 1, "collaboration": 1}

In [24]:
# data=json.load(open("final_ceo_raw_data.json","r"))

In [25]:
# main_df=pd.read_csv("../../../Leadership_51k.csv")
# main_df.head()

In [26]:
# all_website={web:1 for web in main_df["website"]}
# len(all_website)

In [27]:
# df=pd.read_excel("../../../TN Shortlist - Alternate Data.xlsx")
# df.head()

In [28]:
# len({website:1 for website in df["website"]})

In [29]:
# priority_website={}
# notfound={}
# for website in df["website"]:
#     try:
#         priority_website[website]
#     except:
#         try:
#             priority_website[website]=data[website]
         
#         except:
#             notfound[website]=1
# len(priority_website),len(notfound)

total website = 3720 ,
after scraping =2598 (useful) , fail ->1122 
result --> 1319

In [30]:
# data["balasorealloys.com"]

In [31]:
# pd.DataFrame({"website":notfound.keys()}).head(30)

In [32]:

# temp_batch={}
# i=0
# batch_name=0
# for parent,value in priority_website.items():
#     if i!=0 and i%1000==0:
#         json.dump(temp_batch,open(f"Batches2/{batch_name}.json","w"),cls=NumpyEncoder,separators=(', ',': '))
#         print(i%1000)
#         batch_name+=1
#         temp_batch={}
#         i=0
#     temp_batch[parent]=value
#     i+=1